### Imports

In [1]:
import gurobipy as gp
import pandas as pd

### Functions

In [2]:
def extract(data, row, fact, maximize=True):
    text = str(data.iloc[row][fact]).strip("mg%?")
    if str(text) == "nan" or len(text) == 0:
        return maximize * 1_000_000
    return float(text)

### File Input

In [3]:
df = pd.read_excel("Nutritional Facts.xlsx", index_col=0)

### Restaurant / Category Selection

In [12]:
res = ""
cat = ""
real_meal = True
cat_limit = False
subset = df
if res != "":
    subset = subset[subset["Restaurant"] == res]
elif cat != "":
    subset = subset[subset["Category"] == cat]
if real_meal and not cat_limit:
    bad_cats = ["Drinks", "Beverages", "Sauces and Condiments", "Coffee", "Salad Dressing"
                "Ocean Water", "Iced Teas", "Limeades", "Soft Drinks", ]
    subset = subset[~subset["Category"].isin(bad_cats)]
subset  # Contains all food entries

,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,...,Protein,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL
0,Arby's,Sandwiches,Arby's Melt,146g,110,330,12g,4.0g,.5g,35mg,...,18g,18%,20%,12%,39%,13%,8%,36%,2%,https://fastfoodnutrition.org/arbys/arbys-melt
1,Arby's,Sandwiches,Arby-Q Sandwich,182g,100,400,11g,3.5g,.5g,30mg,...,18g,17%,18%,10%,52%,19%,12%,36%,4%,https://fastfoodnutrition.org/arbys/arby-q-san...
2,Arby's,Sandwiches,Beef 'n Cheddar Classic,1 sandwich,180,450,20g,6g,1g,50mg,...,23g,31%,30%,17%,53%,15%,8%,46%,2%,https://fastfoodnutrition.org/arbys/beef-n-che...
3,Arby's,Sandwiches,Beer Battered Fish Sandwich,1 sandwich,210,600,23g,4g,0g,65mg,...,25g,35%,20%,22%,76%,24%,20%,50%,NaN,https://fastfoodnutrition.org/arbys/beer-batte...
4,Arby's,Sandwiches,Buffalo Crispy Chicken Sandwich,1 sandwich,210,500,23g,4.5g,0.0g,55mg,...,24g,35%,23%,18%,78%,16%,16%,48%,NaN,https://fastfoodnutrition.org/arbys/buttermilk...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5948,Zaxby's,Zappetizers,Fried Pickles,?,610,860,69g,9.5g,?,20mg,...,7g,106%,48%,7%,130%,18%,16%,14%,NaN,https://fastfoodnutrition.org/zaxbys/fried-pic...
5949,Zaxby's,Zappetizers,Fried White Cheddar Bites w/ Marinara Sauce,?,260,470,29g,11.0g,?,45mg,...,16g,45%,55%,15%,40%,13%,4%,32%,NaN,https://fastfoodnutrition.org/zaxbys/fried-whi...
5950,Zaxby's,Zappetizers,Onion Rings,?,350,500,40g,5.5g,?,15mg,...,5g,62%,28%,5%,44%,11%,8%,10%,NaN,https://fastfoodnutrition.org/zaxbys/onion-rings
5951,Zaxby's,Zappetizers,Spicy Fried Mushrooms,?,420,620,48g,6.5g,?,15mg,...,8g,74%,33%,5%,58%,14%,20%,16%,NaN,https://fastfoodnutrition.org/zaxbys/spicy-fri...



### Sample Constraint Setting

In [13]:
requirements = {"Protein": 100, "Dietary Fiber": 45, "Vitamin A %": 15}

### Model Building

#### Model & Variables

In [14]:
m = gp.Model()
xis = [m.addVar(vtype=gp.GRB.BINARY) for _ in subset.index]  # GRB.BINARY / GRB.INTEGER

#### Constraints

In [15]:
for fact, req in requirements.items():  # Nutrition Requirements
    print(f"Constraining {fact} >= {req}")
    m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) >= req)

if real_meal:
    for i, food in enumerate(subset.iloc):  # Excludes zero calorie (from fat) entries
        m.addConstr(xis[i] <= extract(subset, i, "Calories From Fat", maximize=True))

Constraining Protein >= 100
Constraining Dietary Fiber >= 45
Constraining Vitamin A % >= 15


#### Objective

In [16]:
m.ModelSense = gp.GRB.MINIMIZE
m.setObjective(sum((x * extract(subset, r, "Calories From Fat", maximize=True) for r, x in enumerate(xis))))

### Solving

In [17]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 5048 rows, 5045 columns and 14860 nonzeros
Model fingerprint: 0xdb94bfd7
Variable types: 0 continuous, 5045 integer (5045 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 2025.0000000
Presolve removed 5045 rows and 1329 columns
Presolve time: 0.23s
Presolved: 3 rows, 3716 columns, 8177 nonzeros
Variable types: 0 continuous, 3716 integer (3216 binary)

Root relaxation: objective 4.857627e+01, 2 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   48.57627    0    2 2025.00000   48.57627  97.6%     -    0s
H    0     0                    1064.0000000   48.57627  95.4%     -    0s
H    0     0                      51.0000000

### Optimal Objective

In [18]:
print(m.ObjVal, "calories from fat")
m.printAttr('X')
best_choices = [i for i, x in enumerate(xis) if x.x > 0]

51.0 calories from fat

    Variable            X 
-------------------------
       C1115            1 
       C1124            1 
       C1965            1 
       C1979            1 
       C2935            1 
       C2968            1 
       C4386            1 


### Corresponding Meal(s)

In [19]:
print(subset.iloc[best_choices]["Food"], ":")
subset = subset.iloc[best_choices]
subset = subset[subset["Protein"] > '0g']
subset = subset[subset["Dietary Fiber"] > '0g']
subset

1391                       Black Beans
1400                       Pinto Beans
2289                       Plain Bagel
2303                    English Muffin
3349                         Baked Cod
3430                      Baked Potato
5055    Fresh Fit Crispy Chicken Salad
Name: Food, dtype: object :


,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,...,Protein,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL
1391,Chipotle,Extras and Toppings,Black Beans,4 oz,9,120,1g,0.0g,0.0g,0mg,...,7g,2%,0%,0%,10%,8%,44%,14%,2%,https://fastfoodnutrition.org/chipotle/black-b...
1400,Chipotle,Extras and Toppings,Pinto Beans,4 oz,9,120,1g,0.0g,0.0g,5mg,...,7g,2%,0%,2%,14%,7%,40%,14%,2%,https://fastfoodnutrition.org/chipotle/pinto-b...
2289,Dunkin Donuts,Bagels,Plain Bagel,?,5,310,1g,0.0g,0.0g,0mg,...,11g,2%,0%,0%,26%,21%,16%,22%,0%,https://fastfoodnutrition.org/dunkin-donuts/pl...
2303,Dunkin Donuts,Bakery Favorites,English Muffin,?,5,140,1g,0.0g,0.0g,0mg,...,4g,2%,0%,0%,5%,10%,24%,8%,0%,https://fastfoodnutrition.org/dunkin-donuts/en...
3430,Long John Silver's,Sides,Baked Potato,1 piece,3,297,0g,0.0g,0.0g,0mg,...,6g,0%,0%,0%,15%,22%,32%,12%,0%,https://fastfoodnutrition.org/long-john-silver...
5055,Subway,Salads,Fresh Fit Crispy Chicken Salad,1 salad,15,460,20g,2g,0g,100mg,...,41g,31%,10%,33%,60%,11%,32%,82%,60%,https://fastfoodnutrition.org/subway/fresh-fit...
